In [ ]:
from datetime import datetime, timedelta


import pandas as pd
import psycopg2 as pg
from tqdm.auto import tqdm
from sqlalchemy import create_engine, URL
from sqlalchemy.sql import text

tqdm.pandas()

In [ ]:
url = URL.create(
    drivername="postgresql+psycopg2",
    username="postgres", password=open("/data/home/username/.pw", "r").read().strip(),
    host="localhost", database="mimic",
    )
engine = create_engine(url=url, connect_args={'options': "--search_path=mimiciii"}, execution_options={"stream_results": True})
conn = engine.connect()

In [ ]:
# need to select the 56 features + summary stats from RoS
icustays = pd.read_sql_query(sql=text("select * from mimiciii.icustays"), con=conn)
patients = pd.read_sql_query(sql=text("select * from mimiciii.patients"), con=conn)
admissions = pd.read_sql_query(sql=text("select * from mimiciii.admissions"), con=conn)

In [ ]:
ce = pd.read_csv("/data4/mimiciii/physionet.org/files/mimiciii/1.4/CHARTEVENTS.csv.gz", low_memory=False)
ce.head()

In [ ]:
items = pd.read_sql_query(sql=text("select * from mimiciii.d_items"), con=conn)
items

In [ ]:
import pandas as pd
sepsis_df = pd.read_csv("/data2/username/sepsis3_mimic_ros_replication.csv")
sepsis_df_no_bc = sepsis_df.dropna(subset="blood_culture_time")
sepsis_df_no_bc.to_csv("/data2/username/sepsis3_mimic_ros_20240123.csv")

In [ ]:
sepsis_df_no_bc["sepsis-3"].mean()

In [ ]:
#sepsis_df = pd.read_csv("../mimic_sepsis/new_sepsis_mimiciii.csv") 
# -- new strat -- left join everything INTO the sepsis DF (maintains original cohort excl criteria)
sepsis_df = pd.read_csv("/data4/username/sepsis3-mimic/data/sepsis3-df-alt.csv")
sepsis_df.head(n=20)

In [ ]:
icustays_metavision = icustays[icustays.dbsource == "metavision"]
#metavision_sepsis = sepsis_df[sepsis_df.icustay_id.isin(icustays_metavision.icustay_id)]
ce_for_sepsis = ce[ce.ICUSTAY_ID.isin(sepsis_df.icustay_id)]

In [ ]:
set(sepsis_df.icustay_id) - set(icustays_metavision.icustay_id) # all sepsis ICU stays in metavision subset

In [ ]:
# confirm sepsis definition

(((sepsis_df["sofa>=2"] == 1) & (sepsis_df["suspicion_poe"])) == sepsis_df["sepsis-3"]).all()

In [ ]:
sepsis_df["intime"] - sepsis_df["suspected_infection_time_poe"]

In [ ]:
sepsis_df.loc[:, "sepsis-3"].mean() # prevalence in metavision pop

In [ ]:
df = sepsis_df.loc[pd.isnull(sepsis_df["suspected_infection_time_poe"]),["antibiotic_time_poe", "suspected_infection_time_poe", "blood_culture_time", "sofa", "sepsis-3"]]


In [ ]:
TIME_FMT = "%Y-%m-%d %H:%M:%S"

def get_onset_hours(row):
    """intime = icustays[icustays.icustay_id == row.icustayid].intime
    first_charttime = ce_for_metavision[ce_for_metavision.ICUSTAY_ID == row.icustayid].CHARTTIME.min()
    sepsis_timestamp = datetime.fromtimestamp(row.sepsis_time)
    onset_from_intime = (sepsis_timestamp - intime.item()).seconds / 3600
    if pd.notnull(first_charttime):
        onset_from_charttime = (sepsis_timestamp - datetime.strptime(first_charttime, TIME_FMT)).seconds / 3600
    else:
        onset_from_charttime = -1 # no chart events were found for such patients -- 2x check
    return onset_from_intime, onset_from_charttime"""
    candidate_onset_time = datetime.strptime(row["suspected_infection_time_poe"], TIME_FMT) # if blood culture was first, it will be eq to suspected infection time
    
    if candidate_onset_time.hour == 0 and candidate_onset_time.minute == 0 and candidate_onset_time.second == 0:
        # unreliable timestamp; use blood culture time if on the same day (i.e., blood culture taken within 24h)
        blood_culture_time = datetime.strptime(row["blood_culture_time"], TIME_FMT)
        if blood_culture_time.year == candidate_onset_time.year and blood_culture_time.month == candidate_onset_time.month and blood_culture_time.day == candidate_onset_time.day:
            candidate_onset_time = blood_culture_time
            # o/w, use the antibiotic time, since the zero-houred timestamp is otherwise closer to the true suspicion of infection time (min of abx and culture time)
    sepsis_onset_time = candidate_onset_time
    intime = datetime.strptime(row["intime"], TIME_FMT)
    first_charttime = datetime.strptime(ce_for_sepsis[ce_for_sepsis.ICUSTAY_ID == row.icustay_id].CHARTTIME.min(), TIME_FMT)    
    onset_from_intime = (sepsis_onset_time - intime).total_seconds() / 3600
    if pd.notnull(first_charttime):
        onset_from_charttime = (sepsis_onset_time - first_charttime).total_seconds() / 3600
    else:
        onset_from_charttime = -1 # no chart events were found for such patients -- 2x check
    return onset_from_intime, onset_from_charttime

results = sepsis_df.loc[sepsis_df["sepsis-3"] == 1].progress_apply(get_onset_hours, axis=1, result_type="expand")
#metavision_sepsis[["from_intime", "from_min_charttime"]] = metavision_sepsis.progress_apply(get_onset_hours, axis=1, result_type="expand")
display(results.head())
#sepsis_df.loc[sepsis_df["sepsis-3"] == 1, ["sepsis_onset_from_intime", "sepsis_onset_from_min_charttime"]] = results

In [ ]:
datetime.strptime(sepsis_df.suspected_infection_time_poe.iloc[3], TIME_FMT).second

In [ ]:
sepsis_df[["antibiotic_time_poe", "blood_culture_time", "suspected_infection_time_poe", "intime", "sepsis_onset_from_min_charttime", "sepsis_onset_from_intime"]]

In [ ]:
sepsis_df.loc[sepsis_df["sepsis-3"] == 1, ["sepsis_onset_from_intime", "sepsis_onset_from_min_charttime"]] = results.values

In [ ]:
final_sepsis_cohort = sepsis_df[((sepsis_df.sepsis_onset_from_min_charttime > 3) & (sepsis_df.sepsis_onset_from_min_charttime <= 11))| (sepsis_df["sepsis-3"] == 0)]
final_sepsis_cohort["sepsis-3"].mean()

In [ ]:
metavision_sepsis_3h = metavision_sepsis[metavision_sepsis.from_min_charttime >= 3]
exclude_ids = metavision_sepsis[metavision_sepsis.from_min_charttime < 3].icustayid

metavision_sepsis_6h = metavision_sepsis[metavision_sepsis.from_min_charttime >= 6]
exclude_ids_6h = metavision_sepsis[(metavision_sepsis.from_min_charttime < 6) | (metavision_sepsis.from_min_charttime > 14)].icustayid

In [ ]:
final_sepsis_cohort

In [ ]:
def get_age(row):
    try:
        return (row.intime - patients[patients.subject_id == row.subject_id].dob.item()).days / 365
    except pd.errors.OutOfBoundsDatetime:
        return 89 # see mimic docs -- this means age >= 89

cohort = final_sepsis_cohort.merge(patients[["subject_id", "gender", "dob"]], on="subject_id", how="left")
cohort = cohort[~cohort.icustay_id.isin(exclude_ids_6h)]

intime_date = pd.to_datetime(cohort['intime']).dt.date
dob_date = pd.to_datetime(cohort['dob']).dt.date
cohort['age'] = (intime_date - dob_date).apply(lambda x: x.days // 365)
cohort.loc[cohort.age == 300, 'age'] = 89


In [ ]:
from functools import partial

TIME_FMT = "%Y-%m-%d %H:%M:%S"

def censor_chartevents(horizon, group):
    horizon_ts = datetime.strptime(group.CHARTTIME.min(), TIME_FMT) + timedelta(hours=horizon)
    return group[group.CHARTTIME <= horizon_ts.strftime(TIME_FMT)]

#ce_censored = ce_for_metavision.groupby("ICUSTAY_ID").progress_apply(partial(censor_chartevents, 6))
ce_censored_sepsis = ce_for_sepsis.groupby("ICUSTAY_ID").progress_apply(partial(censor_chartevents, 3))

In [ ]:
item_counts = ce_censored_sepsis["ITEMID"].value_counts()
items_with_counts = items[items.dbsource == "metavision"].merge(item_counts.to_frame().reset_index(), left_on="itemid", right_on="ITEMID", how="left").drop("ITEMID", axis=1)

In [ ]:
def get_measurements_by_string(name):
    measures = items_with_counts[items_with_counts.label.str.contains(name, case=False, regex=True, na=False)]
    values = ce_censored_sepsis[ce_censored_sepsis.ITEMID.isin(measures.itemid)]
    return values

measures = items_with_counts[items_with_counts.label.str.contains("heart rate", case=False, regex=True, na=False)].sort_values(by="count")
hrs = ce_censored_sepsis[ce_censored_sepsis.ITEMID.isin(measures.itemid)]
measures = items_with_counts[items_with_counts.label.str.contains("systolic", case=False, regex=True, na=False)].sort_values(by="count")
sbps = ce_censored_sepsis[ce_censored_sepsis.ITEMID.isin(measures.itemid)]

In [ ]:
hrs.describe()

In [ ]:

def get_shock_series(group):
    try:
        hr_group = hrs.groupby(level=0).get_group(group.name)
        # merge hr_group and group on charttime
        merged = group.merge(hr_group, on="CHARTTIME", how="inner")
        # hr (right) / sbp (left)
        shock_index = np.where(merged["VALUENUM_x"] != 0, merged["VALUENUM_y"] / merged["VALUENUM_x"], None)
    except KeyError:
        # then shock index cannot be computed -- return null but combine-able thing
        return pd.Series([], name="shock_index")
    return pd.Series(shock_index, name="shock_index")

shock_index_series = sbps.groupby(level=0).progress_apply(get_shock_series)

In [ ]:
from functools import partial, reduce

# max lactic acid
lactic_acid = get_measurements_by_string("lactic")
max_lactic_acid = pd.Series(lactic_acid.groupby(level=0).VALUENUM.max(), name="max_lactic_acid")

# first shock index (hr / sbp) x age
first_shock = pd.Series(shock_index_series.groupby(level=0).first(), name="first_shock_index") # multiply in age later

# last shock index (hr / sbp) x age
last_shock = pd.Series(shock_index_series.groupby(level=0).last(), name="last_shock_index")

# max wbc
wbc = get_measurements_by_string("^wbc$")
max_wbc = pd.Series(wbc.groupby(level=0).VALUENUM.max(), name="max_wbc")

# delta lactic acid
delta_lactic_acid = pd.Series(lactic_acid.groupby(level=0).VALUENUM.apply(lambda x: x.iloc[-1] - x.iloc[0]), name="delta_lactic_acid")

# max neutrophils
neuts = get_measurements_by_string("neuts")
max_neuts = pd.Series(neuts.groupby(level=0).VALUENUM.max(), name="max_neuts")

# max glucose
glucose = get_measurements_by_string("^glucose (finger stick|\([\w\s]+\)$)")
max_glucose = pd.Series(glucose.groupby(level=0).VALUENUM.max(), name="max_glucose")

# max BUN
bun = get_measurements_by_string("^bun$")
max_bun = pd.Series(bun.groupby(level=0).VALUENUM.max(), name="max_bun")

# max RR
rr = get_measurements_by_string("(^RR$|respiratory rate)")
max_rr = pd.Series(rr.groupby(level=0).VALUENUM.max(), name="max_rr")

# last albuminb
albumin = get_measurements_by_string("^albumin$")
last_alb = pd.Series(albumin.groupby(level=0).VALUENUM.last(), name="last_albumin")

# min SBP
sbp = get_measurements_by_string("(blood pressure|bp) systolic")
min_sbp = pd.Series(sbp.groupby(level=0).VALUENUM.min(), name="min_sbp")

# max creat
creat = get_measurements_by_string("^creatinine$")
max_creat = pd.Series(creat.groupby(level=0).VALUENUM.max(), name="max_creatinine")

# max temp F
temp_f = get_measurements_by_string("temperature F")
temp_c = get_measurements_by_string("temperature C")
temp = pd.concat([1.8 * temp_c.VALUENUM + 32, temp_f.VALUENUM], axis=0)
max_temp_f = pd.Series(temp.groupby(level=0).max(), name="max_temp_f")

merge = partial(pd.merge, on='ICUSTAY_ID', how='outer')
initial_covariates = reduce(merge, [
    max_lactic_acid, first_shock, last_shock, max_wbc, delta_lactic_acid,
    max_neuts, max_glucose, max_bun, max_rr, last_alb, min_sbp,
    max_creat, max_temp_f])



In [ ]:
raw_full_df = pd.merge(final_sepsis_cohort, initial_covariates.reset_index(), how="left", left_on="icustay_id", right_on="ICUSTAY_ID")
raw_full_df["first_shock_index_age"] = raw_full_df["first_shock_index"] * raw_full_df["age"] # nans will propagate
raw_full_df["last_shock_index_age"] = raw_full_df["last_shock_index"] * raw_full_df["age"] # nans will propagate
#raw_full_df["sepsis_6h"] = raw_full_df["icustay_id"].isin(metavision_sepsis_6h.icustayid)

In [ ]:
raw_full_df = raw_full_df.merge(metavision_sepsis_6h[["icustayid", "from_intime", "from_min_charttime"]], how="left", left_on="icustay_id", right_on="icustayid")

In [ ]:
raw_full_df.columns

In [ ]:
num_df = raw_full_df.select_dtypes(include=[np.number])
num_df.drop(["excluded", "icustay_id", "hadm_id", "ICUSTAY_ID", "icu_los",
             "hosp_los", "bmi", "sirs", "qsofa", "height", "weight", "elixhauser_hospital",
            "lods"], axis=1).loc[:, (num_df.max(axis=0) != 1) | (num_df.min(axis=0) != 0)].describe()


In [ ]:
num_df.drop(["excluded", "icustay_id", "hadm_id", "ICUSTAY_ID", "icu_los",
             "hosp_los", "bmi", "sirs", "qsofa", "height", "weight", "elixhauser_hospital",
            "lods"], axis=1).isna().sum(axis=0) / len(num_df)

In [ ]:
raw_full_df.select_dtypes(include=[np.number]).max(axis=0)

In [ ]:
features = ["max_lactic_acid", "max_wbc", "delta_lactic_acid", "max_neuts", "max_glucose", "max_bun",        
    "max_rr", "last_albumin", "min_sbp", "max_creatinine", "max_temp_f"]

raw_full_df[features].isna().sum(axis=1).describe()

In [ ]:
raw_full_df[features].isna().sum(axis=0) / len(raw_full_df)

In [ ]:
raw_full_df[["max_lactic_acid", "max_wbc", "delta_lactic_acid", "max_neuts", "max_glucose", "max_bun",        
    "max_rr", "last_albumin", "min_sbp", "max_creatinine", "max_temp_f"]]


In [ ]:
#df_with_ethnicity = raw_full_df.merge(admissions[["subject_id", "ethnicity"]], how="left", on="subject_id").drop_duplicates("icustay_id").reset_index(drop=True)
#df_wb = df_with_ethnicity[df_with_ethnicity.ethnicity.str.startswith("WHITE") | df_with_ethnicity.ethnicity.str.startswith("BLACK")] 
df_wb = raw_full_df[raw_full_df.race_white + raw_full_df.race_black > 0]
df_wb["ethnicity_id"] = (df_wb.race_white == 1).astype(int)
df_wb = df_wb[df_wb[feature_cols].isna().sum(axis=1) < len(feature_cols)]

In [ ]:
sim.df.loc[sim.indices[0], sim.feature_cols].var(axis=0)


In [ ]:
feature_cols = ["max_lactic_acid", "first_shock_index_age", "last_shock_index_age", "max_wbc", "delta_lactic_acid",
               "max_neuts", "max_glucose", "max_bun", "max_rr", "last_albumin", "min_sbp", "max_creatinine", "max_temp_f"]
X = df_wb[feature_cols].fillna(-9999)
y = df_wb["sepsis-3"]
A = df_wb.ethnicity_id

In [ ]:
df_wb.to_csv("/data4/mimiciii/sepsis3_mimic_ros_replication.csv")

In [ ]:

df_wb["sepsis-3"].mean()

In [ ]:
import operator as op
import numpy as np

"""
                        summary_fns=[op.itemgetter(0), op.itemgetter(-1), np.min, np.max, np.mean, lambda x: x[-1] - x[0]],
                        summary_names=["first", "last", "min", "max", "mean", "overall_change"]
"""

def summarize(ce_df):
    value_series = ce_df.VALUENUM
    first, last, minimum, maximum = value_series.iloc[0], value_series.iloc[-1], value_series.min(), value_series.max()
    change_over_time = last - first
    return pd.Series({"first": first, "last": last, "min": minimum, "max": maximum, "delta": change_over_time})

def get_summary_measures(measure, agg_fn):
    measures = items_with_counts[items_with_counts.label.str.contains(measure, case=False, regex=True, na=False)].sort_values(by="count")
    values = ce_for_metavision[ce_for_metavision.ITEMID.isin(measures.itemid)]
    return values.groupby("ICUSTAY_ID").VALUENUM.progress_apply(agg_fn).reset_index()
    #return pd.DataFrame([summarizer(values) for summarizer in summary_fns]

In [ ]:
items_with_counts[items_with_counts.label.str.contains("temperature f", case=False, regex=True, na=False)].sort_values(by="count")


In [ ]:
items_with_counts[items_with_counts.category == "Routine Vital Signs"]

In [ ]:
# abx
"""DOBUTAMINE  (Y/N)
DOPAMINE  (Y/N)
EPINEPHRINE  (Y/N)
NOREPINEPHRINE  (Y/N)"""
def get_icu_stays_for_drug(drugname):
    return rx[rx.drug.str.upper() == drugname.upper()].icustay_id.unique()
    
cohort["dobutamine"] = cohort.icustay_id.isin(get_icu_stays_for_drug("dobutamine")).astype(int)
cohort["dopamine"] = cohort.icustay_id.isin(get_icu_stays_for_drug("dopamine")).astype(int)
cohort["epinephrine"] = cohort.icustay_id.isin(get_icu_stays_for_drug("epinephrine")).astype(int)
cohort["norepinephrine"] = cohort.icustay_id.isin(get_icu_stays_for_drug("norepinephrine")).astype(int)

In [ ]:
ne = pd.read_sql_query(sql=text("select * from mimiciii.noteevents"), con=conn)


In [ ]:
ne_with_ccs

In [ ]:
ne_with_ccs = ne[ne.text.str.contains("CHIEF COMPLAINT") & (ne.category == "Discharge summary")]

In [ ]:
import re

ne_with_ccs["cc_text"] = ne_with_ccs.text.str.extract(r'(?P<cc_text>\s?CHIEF\s+COMPLAINT(?:[\w\s\/]+)?:\s*.+?(?=\n\n))', flags=re.IGNORECASE | re.DOTALL)
